In [5]:
from contents_DB import connect_mongodb, read_from_mongodb

# MongoDB 연결
collection = connect_mongodb('post','all_post')

In [7]:
# DB의 일부를 추출(테스트용)

results_df = read_from_mongodb(collection)
results_df = results_df[:80]

In [8]:
import folium

# 지도 초기화
map = folium.Map(location=[37.5665, 126.9780], zoom_start=12)  # 서울 시청 기준

In [ ]:
from map_utils import extract_road_address, search_place_by_address, coord_info
from map_utils import KAKAO_REST_API_KEY
import json
import requests

# 데이터프레임의 place name과 address를 map에 마킹하기

headers = {"Authorization": f"KakaoAK {KAKAO_REST_API_KEY}"}

# 데이터프레임 순회하며 장소 확인 및 매칭
for idx, row in results_df.iterrows():
    place_name = row['restaurant name']
    print(place_name)
    address = extract_road_address(row['address'])
    print(address)

    # 주소로 좌표 변환 후 매장 이름과 주소 얻기
    result = search_place_by_address(address, place_name)

    if result:
        matched_name = result['place_name']
        matched_address = result['address_name']

        # 매장 이름과 주소를 지도에 마킹
        folium.Marker(
            location=[result['y'], result['x']],
            popup=f"{matched_name} ({matched_address})",
            icon=folium.Icon(color="blue", icon="info-sign")
        ).add_to(map)
        
    else:
        y, x = coord_info(address)
        if y is None or x is None:
            continue

        keyword_url = "https://dapi.kakao.com/v2/local/search/keyword.json"
        keyword_params = {"query": place_name, "x": x, "y": y, "radius": 100}
        keyword_response = requests.get(keyword_url, headers=headers, params=keyword_params)
        if keyword_response.status_code == 200:
            keyword_result = json.loads(keyword_response.text)
            if keyword_result['documents']:
                place_name = keyword_result['documents'][0]['place_name']
                print(f"검색성공 - {place_name}")
                folium.Marker(
                    location=[y, x],
                    popup=f"{place_name} ({address})",
                    icon=folium.Icon(color="blue", icon="info-sign")
                    ).add_to(map)
            else:
                print(f"찾지 못함 - {place_name}")

        else:
            print(f"응답 실패 - '{place_name}'")

print("저장 완료")

순천 풋전집
전라남도 순천시 북부시장길 6
찾지 못함 - 순천 풋전집
알트어스
전라남도 순천시 시민로 24
검색성공 - 아트어스
갤러리안
여수시 돌산읍 강남해안로 78
해미리온
여수시 돌산읍 몰둠벙길 72
검색성공 - 해미리온풀빌라&리조트
플로우브릿지
여수시 소라면 덕양로 412
검색성공 - 카페플로우브릿지
플랜온서면점
부산 부산진구 중앙대로680번가길 38
검색성공 - 플랜온 서면
마쯔오카
부산 중구 중앙대로 2
검색성공 - 마쯔오카 롯데백화점광복점
라온타운
경북 경주시 포석로 1097
검색성공 - 라온타운 경주
용두산 썸머비치
부산 중구 용두산길 37-30
찾지 못함 - 용두산 썸머비치
4233마음센터 광안점
부산 수영구 광남로 148
노블루스트
여수시 돌산읍 몰둠벙길 63
검색성공 - 노블루스트여수풀빌라
커피볶는시골커피양산물금점
양산시 물금읍 범구로 27
검색성공 - 커피볶는 시골커피 양산물금점
다이닝원양산점
양산시 동면 남양산1길 46
검색성공 - 다이닝원 양산점
압구정편의점 서면점
부산진구 중앙대로 680번가길 56
찾지 못함 - 압구정편의점 서면점
타코스타
양산시 물금읍 새실로 159
검색성공 - 타코스타 양산물금점
압구정편의점 서면점
부산진구 중앙대로 680번가길 56
찾지 못함 - 압구정편의점 서면점
한우클라쓰정관점
기장군 정관읍 구연3로 3-36
검색성공 - 한우클라쓰 정관지점
덤톡레스토랑
울주군 서생면 깨목3길 17
베이코드
양산시 물금읍 새실로159
검색성공 - 베이코드
일공공샤브 양산 부산대점
양산시 범구로 14 오슬로파크 1층
검색성공 - 일공공샤브 양산부산대점
한샘리하우스
해운대구 센텀 1로 25
검색성공 - 한샘리하우스 인스타일대리점
샐럽스프부산대점
부산시 금정구 장전온천천로 27
찾지 못함 - 샐럽스프부산대점
누구나홀딱반한닭 경남고성점
고성군 고성읍 동외로 155
각산살롱
경북 구미시 원남로10길 10
카페담
달성군 유가읍 유가사2길 15
신라간장게장
대구 동구 서촌로 78
대구간장게장
대구 동구 서촌로 78
검색성공 - 신라간장게

In [12]:
# HTML 파일로 저장
map.save("matched_places_map.html")
print("지도 생성 완료. 파일명 : matched_places_map.html")

지도 생성 완료. 파일명 : matched_places_map.html
